## Flux Fine Tuning Settings and Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ostris/ai-toolkit.git
%cd ai-toolkit
!git submodule update --init --recursive

In [ ]:
!ls -al
!pip install -qU torch
!pip install -qU huggingface_hub
!pip install -qr requirements.txt

## YAML PATH CHECK

In [ ]:
import os


FLUX_IMG_DIR = os.path.join("/content/drive/MyDrive", 'FLUX DATASET')
TRAINING_YAML_PATH = os.path.join("/content/drive/MyDrive", "FLUX_YAML")


TEST_DATASET_THREE_PING_FLORENCE = os.path.join(FLUX_IMG_DIR, 'TEST_DATASET_THREE_PING_FLORENCE')
TEST_YAML_FLORENCE = os.path.join(TRAINING_YAML_PATH, "test_three_ping_florence.yaml")
print(TEST_DATASET_THREE_PING_FLORENCE)
print(TEST_YAML_FLORENCE)

TEST_DATASET_THREE_PING = os.path.join(FLUX_IMG_DIR, 'TEST_DATASET_THREE_PING')
TEST_YAML = os.path.join(TRAINING_YAML_PATH, "test_three_ping.yaml")
print(TEST_DATASET_THREE_PING)
print(TEST_YAML)

/content/drive/MyDrive/FLUX DATASET/TEST_DATASET_THREE_PING_FLORENCE
/content/drive/MyDrive/FLUX_YAML/test_three_ping_florence.yaml
/content/drive/MyDrive/FLUX DATASET/TEST_DATASET_THREE_PING
/content/drive/MyDrive/FLUX_YAML/test_three_ping.yaml


## HuggingFace Hub Settings

In [ ]:
!pip install -qU huggingface_hub

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import login

HF_TOKEN_WRITE = userdata.get('HF_TOKEN_WRITE')
login(token=HF_TOKEN_WRITE)

## Fine-Tuning Start! - with yaml file

In [ ]:
# Copy yaml file from Googel Drive to ai-toolkit/confing/
!cp /content/drive/MyDrive/FLUX_YAML/test_three_ping.yaml config/test_three_ping.yaml
# !cp /content/drive/MyDrive/FLUX_YAML/test_three_ping_florence.yaml config/test_three_ping_florence.yaml
%cd config
!ls -al
%cd ..

/content/ai-toolkit/config
total 20
drwxr-xr-x  3 root root 4096 Nov 28 07:36 .
drwxr-xr-x 17 root root 4096 Nov 28 07:35 ..
drwxr-xr-x  3 root root 4096 Nov 28 07:35 examples
-rw-------  1 root root 4732 Nov 28 07:36 test_three_ping.yaml
/content/ai-toolkit


In [ ]:
!cat config/test_three_ping.yaml
# !cat config/test_three_ping_florence.yaml

---
job: extension
config:
  # 파일명 선정
  name: "IAM_THREE_PING_TEST_V1"
  process:
    - type: 'sd_trainer'
      # session, sample, 가중치를 저장할 루트 폴더 경로 지정
      training_folder: "output"
      # N번의 한번씩 성능을 확인하려면 주석을 해제
      performance_log_every: 250
      device: cuda:0
      # Trigger Word가 정해졌으면 학습정보의 Caption(사진 설명)에 [trigger]를 추가하면 정해진 Trigger Word로 바뀜
      # 만약에 없다면은 학습정보의 Caption(사진 설명)에 자동으로 추가됨
      trigger_word: "IATPT"
      network:
        type: "lora"
        linear: 16
        linear_alpha: 16
      save:
        dtype: float16 # 저장할 때의 데이터 형태
        save_every: 250 # 저장 주기
        max_step_saves_to_keep: 4 # 얼마나 보유할 지
        push_to_hub: false # true로 바꾼다면 HuggingFace에 업로드 됩니다.
        # 여기에 Hugging Face의 write권한이 있는 토큰과 저장소 정보를 작성하세요 안그러면 입력하라고 뜹니다.
        # hf_repo_id: jwywoo/test_three_ping
        # hf_private: true #whether the repo is private or public
      datasets:
        # 데이터셋은 학습할 사진이 들어간 폴더입니다. 사진에 대한 설명의 경우 txt  파일로 사진파일의 이름과 일치해서 저장해야합니다.
        # 예

In [ ]:
# Initiate fine-tuning
# python run.py config/whatever_yaml_file.yaml
!python run.py config/test_three_ping.yaml
# !python run.py config/test_three_ping_florence.yaml

Running 1 job
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-11-28 07:37:26.364092: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 07:37:26.380589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 07:37:26.401440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 07:37:26.

## Fine-tuning with Gradio UI

In [ ]:
!ls -al
%cd /content/ai-toolkit

In [ ]:
!python flux_train_ui.py

# HuggingFace Uploads
- Google Drive First
- HuggingFace Uploads

## Creating Repo on HuggingFace Hub

In [ ]:
from huggingface_hub import create_repo

repo_name = "iam-three-ping-test-v1"
create_repo(repo_name, repo_type="model", private=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RepoUrl('https://huggingface.co/jwywoo/iam-three-ping-test-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='jwywoo/iam-three-ping-test-v1')

## HuggingFace Uploads - Files

In [ ]:
import os

DRIVE_PATH = os.path.join("/content/drive", "MyDrive")
FLUX_YAML = os.path.join(DRIVE_PATH, "FLUX_YAML")
FINE_TUNED_MODELS = os.path.join(FLUX_YAML, "fine_tuned_models")
THREE_PING_TEST_DIR = os.path.join(FINE_TUNED_MODELS, "IAM_THREE_PING_TEST_V1")
print(THREE_PING_TEST_DIR)

if not os.path.exists(THREE_PING_TEST_DIR):
  print("Path doesn't exist!")
print(os.listdir(THREE_PING_TEST_DIR))

/content/drive/MyDrive/FLUX_YAML/fine_tuned_models/IAM_THREE_PING_TEST_V1
['config.yaml', 'IAM_THREE_PING_TEST_V1_000000250.safetensors', 'IAM_THREE_PING_TEST_V1.safetensors', 'optimizer.pt', 'samples']


In [ ]:
from huggingface_hub import upload_file

repo_name = "jwywoo/iam-three-ping-test-v1"

print(repo_name)

files_uploading = ['IAM_THREE_PING_TEST_V1_000000250.safetensors', 'IAM_THREE_PING_TEST_V1.safetensors']
for file_name in files_uploading:
  path_to_file = os.path.join(THREE_PING_TEST_DIR, file_name)
  upload_file(
      path_or_fileobj=path_to_file,
      path_in_repo=file_name,
      repo_id=repo_name,
      repo_type="model",
      commit_message=f"Upload {file_name}"
  )

jwywoo/iam-three-ping-test-v1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


IAM_THREE_PING_TEST_V1_000000250.safetensors:   0%|          | 0.00/172M [00:00<?, ?B/s]

IAM_THREE_PING_TEST_V1.safetensors:   0%|          | 0.00/172M [00:00<?, ?B/s]

## HuggingFace Uploads - Folders


In [ ]:
from huggingface_hub import upload_folder

folder_path = os.path.join(THREE_PING_TEST_DIR, "samples")
repo_name = "jwywoo/iam-three-ping-test-v1"

upload_folder(
    folder_path=folder_path,
    repo_id=repo_name,
    repo_type="model",
    path_in_repo = "samples/",
    commit_message=f"Upload samples"
)



CommitInfo(commit_url='https://huggingface.co/jwywoo/iam-three-ping-test-v1/commit/f30bdd1b195f5fee0804dc495f17fd886b8198a0', commit_message='Upload samples', commit_description='', oid='f30bdd1b195f5fee0804dc495f17fd886b8198a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jwywoo/iam-three-ping-test-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='jwywoo/iam-three-ping-test-v1'), pr_revision=None, pr_num=None)